# Week 11 Coding Homework – Event Studies & Placebo Tests

In [2]:
!pip install statsmodels

import numpy as np
import statsmodels.api as sm

# For reproducibility in ad-hoc experiments
np.random.seed(0)

num = 1000
event_time = int(num / 2)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 12.2 MB/s  0:00:00m0:00:01:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]


In [3]:
num = 1000 
 
event_time = int(num / 2) 
 
R_market = np.random.normal(0, 1, num) + np.arange(num) / num 
 
R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == int(num / 2) + 1) * 2 
 
results = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit() 
 
alpha, beta = results.params 
 
resid = R_target - results.predict(sm.add_constant(R_market)) 
 
print(resid[event_time + 1] / resid[:event_time].std(ddof = 2)) 

1.9268039244154727


## Question 1 

Which is closest to the probability that this t-test will be able to detect the event at event_time + 1 with the given code? 

In [4]:
def simulate_q1_once(num=1000):
    event_time = int(num / 2)
    
    # Generate data
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num
    R_target = (
        2
        + R_market
        + np.random.normal(0, 1, num)
        + (np.arange(num) == int(num / 2) + 1) * 2
    )
    
    # Fit market model before event_time
    results = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit()
    
    # Residuals over full horizon
    resid = R_target - results.predict(sm.add_constant(R_market))
    
    # t-stat at event_time + 1
    t_val = resid[event_time + 1] / resid[:event_time].std(ddof=2)
    return t_val

# Run many simulations
np.random.seed(0)
num_sims = 2000
t_values = [simulate_q1_once(num) for _ in range(num_sims)]

# Estimate power (probability of detection)
power_estimate = np.mean(np.abs(t_values) > 1.96)
power_estimate


np.float64(0.5035)

## Question 2 

Use the same code but put np.random.seed(0) at the beginning of each loop to ensure that you are performing placebo tests on a fixed dataset. Perform a placebo test by setting the fictitious event_time to all possible times, while leaving the event in R_target at just the 1 time. The placebo test trains itself on the data leading up to the fictitious event. About what fraction of placebo tests seem to detect an event at the fictitious event time? 


In [5]:
# Generate one fixed dataset
np.random.seed(0)
R_market = np.random.normal(0, 1, num) + np.arange(num) / num
R_target = (
    2
    + R_market
    + np.random.normal(0, 1, num)
    + (np.arange(num) == event_time + 1) * 2  # real event at event_time+1
)

# Perform placebo tests for all plausible fictitious event times
placebo_results = []

# Avoid too-small samples at the beginning and end
for fict_event in range(10, num - 10):
    # Train on data BEFORE the fictitious event
    R_m_train = R_market[:fict_event]
    R_t_train = R_target[:fict_event]
    
    model = sm.OLS(R_t_train, sm.add_constant(R_m_train)).fit()
    
    # Residuals on full series
    resid = R_target - model.predict(sm.add_constant(R_market))
    
    # t-stat at fictitious event
    t_stat = resid[fict_event] / resid[:fict_event].std(ddof=2)
    
    placebo_results.append(np.abs(t_stat) > 1.96)

fraction_placebo_detect = np.mean(placebo_results)
fraction_placebo_detect


np.float64(0.04795918367346939)

## Question 3 

Do the same placebo test, but this time only run the test 20 times before and twenty times after the actual event. On average (over many runs of the code), what fraction of the 40 placebo tests get a higher t-value than the actual event? This time, adjust np.random.seed() to represent a different dataset when needed. 

In [6]:
def simulate_q3_once(num=1000, seed=0):
    np.random.seed(seed)
    event_time = int(num / 2)
    
    # Generate dataset
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num
    R_target = (
        2
        + R_market
        + np.random.normal(0, 1, num)
        + (np.arange(num) == event_time + 1) * 2  # real event
    )
    
    # Fit on data before the true event_time (as in Q1)
    results = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit()
    resid = R_target - results.predict(sm.add_constant(R_market))
    
    # True event t-stat
    t_true = resid[event_time + 1] / resid[:event_time].std(ddof=2)
    
    # Placebo events: 20 before and 20 after the true event
    placebo_ts = []
    for offset in range(-20, 0):
        fict_event = event_time + 1 + offset
        t_val = resid[fict_event] / resid[:fict_event].std(ddof=2)
        placebo_ts.append(t_val)
    for offset in range(1, 21):
        fict_event = event_time + 1 + offset
        t_val = resid[fict_event] / resid[:fict_event].std(ddof=2)
        placebo_ts.append(t_val)
    
    placebo_ts = np.array(placebo_ts)
    frac_larger = np.mean(np.abs(placebo_ts) > np.abs(t_true))
    return frac_larger

# Run many simulations with different seeds
num_runs = 300
fractions = [simulate_q3_once(num=num, seed=seed) for seed in range(num_runs)]
np.mean(fractions)


np.float64(0.14575000000000002)

In [7]:
def make_error(corr_const, num): 
 
 
  sigma = 5 * 1 / np.sqrt((1 - corr_const)**2 / (1 - corr_const**2)) 
 
 
  err = list() 
 
 
  prev = np.random.normal(0, sigma) 
 
 
  for n in range(num): 
 
 
    prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, sigma) 
 
 
    err.append(prev) 
 
 
  return np.array(err) 

## Question 4 

Do the same thing as in question 2, but this time use make_error with corr_const = 0.9 to generate the error for R_target instead of np.random.normal. Consider before attempting this: Would you expect this kind of dataset, where errors are not independent, to result in more or fewer false positives in the placebo tests? 

In [8]:
def make_error(corr_const, num):
    sigma = 5 * 1 / np.sqrt((1 - corr_const)**2 / (1 - corr_const**2))
    err = []
    prev = np.random.normal(0, sigma)
    for n in range(num):
        prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, sigma)
        err.append(prev)
    return np.array(err)

# Generate one fixed dataset with autocorrelated errors
np.random.seed(0)
R_market = np.random.normal(0, 1, num) + np.arange(num) / num
err = make_error(0.9, num)

R_target_auto = (
    2
    + R_market
    + err
    + (np.arange(num) == event_time + 1) * 2  # real event at event_time+1
)

# Placebo tests over many fictitious event times
placebo_results_auto = []

for fict_event in range(10, num - 10):
    # Train on data before fictitious event
    R_m_train = R_market[:fict_event]
    R_t_train = R_target_auto[:fict_event]
    
    model = sm.OLS(R_t_train, sm.add_constant(R_m_train)).fit()
    
    resid = R_target_auto - model.predict(sm.add_constant(R_market))
    
    t_stat = resid[fict_event] / resid[:fict_event].std(ddof=2)
    placebo_results_auto.append(np.abs(t_stat) > 1.96)

fraction_placebo_detect_auto = np.mean(placebo_results_auto)
fraction_placebo_detect_auto


np.float64(0.04081632653061224)

## REFLECTION

1. Construct a dataset for an event study where the value, derivative, and second derivative of a trend all change discontinuously (suddenly) after an event.
Build a model that tries to decide whether the event is real (has a nonzero effect) using:
(a) only the value,

In [9]:
import numpy as np
import statsmodels.api as sm

np.random.seed(0)

# Set up time and event
T = 200
t = np.arange(T)
event_time = 100
post = (t > event_time).astype(int)

# Construct Y with different level, slope, and curvature after the event
# Pre-event: mild quadratic
# Post-event: bigger intercept, steeper slope, more curvature
Y = np.where(
    t <= event_time,
    0.01 * t**2 + 0.1 * t,                       # pre-event trend
    (0.03 * t**2 + 0.6 * t + 10),                # post-event trend: level, slope, curvature all changed
)

# Add noise
Y = Y + np.random.normal(0, 1, size=T)

# Model using ONLY the value (no explicit derivatives),
# but allowing for a general trend via t and t^2
X = np.column_stack([post, t, t**2])   # event indicator + time + time^2
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
print(model.summary())

# The key question: is the event "real"?
# That is, is the coefficient on `post` (the event dummy) significantly nonzero?
beta_event = model.params[1]
pval_event = model.pvalues[1]
print("Event coefficient (beta_event):", beta_event)
print("p-value for event effect:", pval_event)


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 3.715e+04
Date:                Sat, 22 Nov 2025   Prob (F-statistic):          9.94e-270
Time:                        22:49:33   Log-Likelihood:                -860.88
No. Observations:                 200   AIC:                             1730.
Df Residuals:                     196   BIC:                             1743.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         44.5678      3.992     11.164      0.0

(b) the value, derivative, and second derivative.

For part (b), I added the first and second differences so the model could look at how the value, slope, and curvature all changed after the event. This version did a better job because it did not just say something happened, it showed what changed. It picked up the jump in the value, the shift in the trend, and the change in curvature separately. Here, “better” means it detected the event more clearly and gave a more useful picture of what actually changed instead of blending everything into one effect.

## Which of these models is better at detecting and/or quantifying the impact of the event?  (What might "better" mean here?)

The model that uses the value, the first difference, and the second difference is better because it captures more of the ways the event affects the data. Instead of mixing everything into one shift, it shows whether the event changed the level, the slope, or the curvature. In this context, “better” means the model is more sensitive to the event, offers clearer evidence that something changed, and gives a more detailed picture of what the impact actually looks like.

2. Construct a dataset in which there are three groups whose values each increase discontinuously (suddenly) by the same amount at a shared event; they change in parallel
over time, but they have different starting values. 

For this part, I set up a simple panel-style dataset with three groups observed over time. Each group followed the same upward trend over time, but I gave them different starting values by adding a different constant to each group so one started low, one in the middle, and one higher. At a chosen event time, I added the same fixed jump to all three groups’ values so they each increased suddenly by the same amount. This way the groups stayed in parallel before and after the event, kept their baseline differences, and shared a common discrete jump at the event.

Create a model that combines group fixed effects with an event study, as suggested in the online reading.
Explain what you did, how the model works, and how it accounts for both baseline differences and the common event effect.

I fit a regression that included group fixed effects along with a post-event indicator. The group fixed effects captured the baseline differences between the three groups, since each group started at a different level but followed the same trend. The post-event indicator captured the sudden jump that all groups experienced at the same time. By separating these two pieces, the model explained who starts higher through the fixed effects and explained the shared event impact through the post variable. This let the model account for both the different starting points across groups and the common shift caused by the event.